In [ ]:
# Platform configs
url = "http://localhost:4000"
project_id = "660ab0b7c4368d2cd7e713eb"

project_setup = {
    "url":  url,
    "project_id": project_id
}

In [ ]:
checkpoint = "M4-ai/tau-0.5B"
device = "cpu"

model_setup = {
    "checkpoint": checkpoint,
    "model_type": "dec",
    "embeddings": "model.embed_tokens"
}

run_config = {
    "device": device,
    "max_new_tokens": 100,
    # "temperature": 1, # Note that if you add this, then you need to add "do_sample": True
    # "do_sample": True
}

In [ ]:
!pip install datasets -q

In [ ]:
from datasets import load_dataset

dataset = load_dataset("truthful_qa", 'multiple_choice')

In [ ]:
dataset["validation"][1]

In [ ]:
import llmex

In [ ]:
model = llmex.from_pretrained(project_setup=project_setup, model_setup=model_setup, run_config=run_config)

In [ ]:
def prompter(question, choices):
    return f"""Please give the correct answer to the following multiple choice question:
    {question}
    The choices are: 
    {choices}
    What is the answer? """

def datapoint_to_labels(choices, labels):
    answers = []
    for choice, label in zip(choices, labels):
        if (label == 1):
            answers.append(choice)

    return ",".join(answers) 
    
def datapoint_to_prompt(point):
    question = point.get("question")
    targets = point.get("mc1_targets")
    target_choices = targets.get("choices")
    choices = ",".join(target_choices) 
    labels = targets.get("labels")
    
    prompt = prompter(question, target_choices)
    answers = datapoint_to_labels(target_choices, labels)
    
    return prompt, answers
    
prompt, answer = datapoint_to_prompt(dataset["validation"][1])
prompt, answer

In [ ]:
result = model.predict(prompt, generate_explanations=False, max_new_tokens=50)
result